In [1]:
# install replicate client
!pip install replicate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [4]:
# get a token: https://replicate.com/account

from google.colab import userdata
import os

api_key = userdata.get("REPLICATE_API_KEY")
os.environ["REPLICATE_API_TOKEN"] = api_key

In [9]:
# run the model

import replicate

prompt = "a guy sitting in a foodball ground"

output = replicate.run(
  "stability-ai/sdxl:39ed52f2a78e934b3ba6e2a89f5b1c712de7dfea535525255b1aa35c5565e08b",
  input={"prompt": prompt}
)

output

['https://replicate.delivery/pbxt/XOlOxeAmdyxaaaMfcyDC7fAeKf7rS6DuwoIhrReeYFrVwUqHJA/out-0.png']

In [10]:
# viwing in notebook

from IPython.display import Image
Image(url=output[0])

In [11]:
# another model

image = output[0]
prompt = "what is in the picture ?"

replicate.run(
  "andreasjansson/blip-2:4b32258c42e9efd4288bb9910bc532a69727f9acd26aa08e175713a0a857a608",
  input={"image": output[0], "prompt": prompt}
)

'a soccer stadium'

In [18]:
# run a model in the baground

model = replicate.models.get("meta/musicgen")
version = model.versions.get("b05b1dff1d8c6dc63d14b0cdb42135378dcb87f6373b0d3d341ede46e59e2b38")
prompt = "a motivating audio"
prediction = replicate.predictions.create(
    version=version,
    input={"prompt": prompt})

In [22]:
prediction.status

'starting'

In [14]:
prediction

Prediction(id='bmljlatbmqaa7h2ejmyn4oadoa', model='meta/musicgen', version='b05b1dff1d8c6dc63d14b0cdb42135378dcb87f6373b0d3d341ede46e59e2b38', status='starting', input={'prompt': 'Edo25 major g melodies that sound triumphant and cinematic. Leading up to a crescendo that resolves in a 9th harmonic'}, output=None, logs='', error=None, metrics=None, created_at='2024-01-23T10:46:27.962844496Z', started_at=None, completed_at=None, urls={'cancel': 'https://api.replicate.com/v1/predictions/bmljlatbmqaa7h2ejmyn4oadoa/cancel', 'get': 'https://api.replicate.com/v1/predictions/bmljlatbmqaa7h2ejmyn4oadoa'})

In [23]:
# predicting

prediction.wait()

In [24]:
prediction.reload()

In [25]:
prediction.status

'succeeded'

In [26]:
# showing

import IPython
IPython.display.Audio(prediction.output)

In [27]:
# Run a model in the background and get a webhook

model = replicate.models.get("jd7h/photomaker")
version = model.versions.get("79b21a43649e0d08a7cbf9b553420047456de4492efb1a7c73d500c1c8812fc6")
prediction = replicate.predictions.create(
    version=version,
    input={
        "seed": 1143488585,
        "prompt": "A photo of a scientist winning the Nobel prize img",
        "num_steps": 50,
        "style_name": "Photographic (Default)",
        "input_image": "https://replicate.delivery/pbxt/KF73Yow3WJw7ebPRq1tPohZXN0bjoAjbNcXiJzeineGGAv1A/newton_0.jpg",
        "num_outputs": 1,
        "guidance_scale": 5,
        "negative_prompt": "nsfw, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry",
        "style_strength_ratio": 20
    },
    webhook="https://example.com/your-webhook",
    webhook_events_filter=["completed"]
)

In [29]:
prediction

Prediction(id='oibufftbbgl76cbxksxejedqaa', model='tencentarc/photomaker', version='79b21a43649e0d08a7cbf9b553420047456de4492efb1a7c73d500c1c8812fc6', status='starting', input={'guidance_scale': 5, 'input_image': 'https://replicate.delivery/pbxt/KF73Yow3WJw7ebPRq1tPohZXN0bjoAjbNcXiJzeineGGAv1A/newton_0.jpg', 'negative_prompt': 'nsfw, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry', 'num_outputs': 1, 'num_steps': 50, 'prompt': 'A photo of a scientist winning the Nobel prize img', 'seed': 1143488585, 'style_name': 'Photographic (Default)', 'style_strength_ratio': 20}, output=None, logs='', error=None, metrics=None, created_at='2024-01-23T10:51:24.634000966Z', started_at=None, completed_at=None, urls={'cancel': 'https://api.replicate.com/v1/predictions/oibufftbbgl76cbxksxejedqaa/cancel', 'get': 'https://api.replicate.com/v1/predictions/oibuf

In [30]:
# compose model into a pipeline

# the models
image_get = replicate.models.get("stability-ai/sdxl").versions.get("39ed52f2a78e934b3ba6e2a89f5b1c712de7dfea535525255b1aa35c5565e08b")
enhance_img = replicate.models.get("nightmareai/real-esrgan").versions.get("42fed1c4974146d4d2414e2be2c5277c7fcf05fcc3a73abf41610695738c1d7b")

In [31]:
# run pipeline

image = replicate.predictions.create(version=image_get, input={"prompt": "avocado armchair"})
image.wait()
image_out = image.output[0]

upscaled_image = replicate.predictions.create(version=enhance_img, input={"image": image_out})
upscaled_image.wait()

In [32]:
# seeing the enhanced image

from IPython.display import Image
Image(url=upscaled_image.output)

In [35]:
# Get streaming output from a running model

iterator = replicate.run(
  "mistralai/mixtral-8x7b-instruct-v0.1",
  input={"prompt": "Who was Dolly the sheep?"},
)

for text in iterator:
      print(text)




🐑
 D
olly
 the
 sheep
 was
 the
 first
 mamm
al
 to
 be
 successfully
 cl
oned
 from
 an
 adult
 cell
.
 She
 was
 born
 on
 July
 
5
,
 
1
9
9
6
,
 and
 lived
 her
 life
 at
 the
 Ros
lin
 Institute
 in
 Scotland
.
 D
olly
'
s
 creation
 was
 a
 ground
breaking
 scientific
 achievement
,
 as
 it
 demonstrated
 the
 possibility
 of
 cl
oning
 mamm
als
 using
 a
 technique
 called
 som
atic
 cell
 nuclear
 transfer
.




The
 process
 of
 creating
 D
olly
 involved
 removing
 the
 nucle
us
 from
 an
 egg
 cell
 and
 replacing
 it
 with
 the
 nucle
us
 from
 a
 cell
 taken
 from
 the
 m
am
mary
 g
land
 of
 a
 adult
 sheep
.
 The
 recon
struct
ed
 egg
 was
 then
 stim
ulated
 to
 divide
 and
 grow
 into
 an
 emb
ry
o
,
 which
 was
 impl
anted
 into
 a
 sur
ro
gate
 mother
.
 D
olly
 was
 the
 result
 of
 this
 process
,
 and
 her
 birth
 proved
 that
 specialized
 cells
 could
 be
 used
 to
 create
 an
 exact
 genetic
 copy
 of
 the
 original
 animal
.




D
olly
'
s
 creation
 spark


In [36]:
# save output files

from urllib.request import urlretrieve

model = replicate.models.get("stability-ai/stable-video-diffusion")
version = model.versions.get("3f0457e4619daac51203dedb472816fd4af51f3149fa7a9e0b5ffcf1b8172438")

prediction = replicate.predictions.create(
    version=version,
    input={
      "cond_aug": 0.02,
      "decoding_t": 7,
      "input_image": "https://replicate.delivery/pbxt/JvLi9smWKKDfQpylBYosqQRfPKZPntuAziesp0VuPjidq61n/rocket.png",
      "video_length": "14_frames_with_svd",
      "sizing_strategy": "maintain_aspect_ratio",
      "motion_bucket_id": 127,
      "frames_per_second": 6
    })
prediction.wait()

urlretrieve(prediction.output, "/tmp/out.mp4")

('/tmp/out.mp4', <http.client.HTTPMessage at 0x7ff3567ebb20>)

In [37]:
from IPython.display import Image
Image(url=upscaled_image.output)

That's it, thanks a lottt ♥